## 逻辑回归

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

/home/mj/.conda/envs/ten/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
##1.读取数据
mnist = input_data.read_data_sets('./data/mnist',one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
#2.查看数据维度
mnist.train.images.shape

(55000, 784)

In [4]:
#3.查看target维度
mnist.train.labels.shape

(55000, 10)

In [5]:
batch_size = 128
X = tf.placeholder(tf.float32,[batch_size,784],name='X_placeholder')
Y = tf.placeholder(tf.int32,[batch_size,10],name='Y_placeholder')

In [6]:
w = tf.Variable(tf.random_normal(shape=[784,10],stddev=0.01),name='weights')
b = tf.Variable(tf.zeros([1,10],name='bias'))

In [7]:
logits = tf.matmul(X,w)+b#拿到每个类别的score

In [8]:
##计算多分类softmax的loss function
#求交叉熵损失,直接从得分和y得到交叉熵
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y,name='loss')

#求平均
loss = tf.reduce_mean(entropy)

In [9]:
learning_rate = 0.01
#adam不保证全局最小点
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [22]:
n_epochs = 30
with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs/logistic_reg',sess.graph)
    
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs):
        total_loss = 0
        for _ in range(n_batches):
            X_batch,Y_batch = mnist.train.next_batch(batch_size)
            _,loss_batch = sess.run([optimizer,loss],feed_dict={X:X_batch,Y:Y_batch})
            total_loss += loss_batch
        print ('Average loss epoch{0}:{1}'.format(i,total_loss/n_batches))
        
    print ('Total time:{0} second'.format(time.time() - start_time))
    print ('Optimaization Finished')
    
    #测试模型
    preds = tf.nn.softmax(logits)
    correct_preds = tf.equal(tf.argmax(preds,1),tf.argmax(Y,1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds,tf.float32))
    
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch,Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run([accuracy],feed_dict={X:X_batch,Y:Y_batch})
        total_correct_preds += accuracy_batch[0]
    
    print ('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))
    
    writer.close()
    
    

Average loss epoch0:0.3659602535285038
Average loss epoch1:0.29102559082019025
Average loss epoch2:0.28658302331512625
Average loss epoch3:0.2779349525630613
Average loss epoch4:0.2726540446698249
Average loss epoch5:0.2797471414600219
Average loss epoch6:0.26561596314042873
Average loss epoch7:0.2712698404640307
Average loss epoch8:0.2641519125559669
Average loss epoch9:0.26742032100826435
Average loss epoch10:0.26415419971053694
Average loss epoch11:0.26020271827777225
Average loss epoch12:0.26169167196764537
Average loss epoch13:0.2625817969128802
Average loss epoch14:0.26130288257534967
Average loss epoch15:0.2553508942186971
Average loss epoch16:0.26190199612668064
Average loss epoch17:0.250824469195935
Average loss epoch18:0.2566128178111045
Average loss epoch19:0.2524394335134046
Average loss epoch20:0.2635448021696998
Average loss epoch21:0.2534775614286914
Average loss epoch22:0.2565439400938285
Average loss epoch23:0.2553741223154924
Average loss epoch24:0.25304816843885364
A